# Tutorial 1 - WaveBot
The goal of this tutorial is to familiarize new users with how to set up and run optimization problems using WecOptTool. 
It uses a one-body WEC, the WaveBot, in one degree of freedom in regular waves. 

![WaveBot Photo](https://live.staticflickr.com/65535/51855905347_de87ccaaba_z.jpg)

At the end of this tutorial the user will perform control co-design of the WEC's geometry and a corresponding optimal controller to maximize electrical power. 
We build up to this problem in three parts of successive complexity:

1. [Optimal control for maximum mechanical power](#1.-Optimal-control-for-maximum-mechanical-power)
2. [Optimal control for maximum electrical power](#2.-Optimal-control-for-maximum-electrical-power)
3. [Control co-design of the WEC's geometry for maximum electrical power](#3.-Control-co-design-of-the-WEC-geometry-for-maximum-electrical-power)

We will start by loading the necessary modules: 

* Import Autograd (wrapper on NumPy, required) for automatic differentiation
* Import other packages we will use in this tutorial 
* Import WecOptTool 

In [1]:
import autograd.numpy as np
import capytaine as cpy
import matplotlib.pyplot as plt
from scipy.optimize import brute

import wecopttool as wot


## 1. Optimal control for maximum mechanical power
This example illustrates how to set up, run, and analyze a basic optimization problem within WecOptTool.

The objective of this example is to **find the optimal PTO force time-series** that produces the most mechanical power subject to the WEC dynamics and a maximum force the PTO can exert.

WecOptTool requires the following to be defined to successfully run its optimization routines:
- The WEC object, including all of its properties and constraints
- The wave condition
- The objective function

<div>
<img src="https://live.staticflickr.com/65535/52435098523_37d6a2ca94_k.jpg" width="1000">
</div>

The graphic shows all the requirements for this first part of the tutorial: from the wave on the left, to the objective (mechanical power) on the right.
The WEC object, with all it's components, is illustrated in the middle. The components inside the blue box are the WEC properties that are actually passed on to the optimizer.
In short, the WEC's hydrodynamic properties are modelled by
1. Defining the WEC's geometry
2. Meshing the geometry
3. Obtaining the WEC's BEM cofficients based on the mesh
4. Determening the WEC's intrinsic impedance model based on the BEM coefficients

For this first part of the tutorial, the heave-only, WEC-PTO kinematics are trivial (Unity) and the PTO is assumed to be lossless.

### WEC object
In this section we will create the `WEC` object, which contains all the information about the WEC and its dynamics. This constitutes the vast majority of the setup required to run WecOptTool.

Our `WEC` object requires information about the mesh, degrees of freedom, mass and hydrostatic properties, linear hydrodynamic coefficients (from a BEM solution), any additional dynamic forces (e.g. PTO force, mooring, non-linear hydrodynamics), and constraints (e.g. maximum PTO extension). 
In this case, the only additional force will be the PTO force and the only constraint will be a maximum PTO force of $2,000 N$.

#### Mesh
First, we will create a surface mesh for the hull and store it using the `FloatingBody` object from Capytaine. The WaveBot mesh is pre-defined in the `wecopttool.geom` module, so we will call it directly from there. We will only model the heave degree of freedom in this case. Note that the Capytaine `from_meshio` method can also import from other file types (STL, VTK, MSH, etc.)

In [2]:
wb = wot.geom.WaveBot()  # use standard dimensions
mesh_size_factor = 0.5 # 1.0 for default, smaller to refine mesh
mesh = wb.mesh(mesh_size_factor)
fb = cpy.FloatingBody.from_meshio(mesh, name="WaveBot")
fb.add_translation_dof(name="Heave")
ndof = fb.nb_dofs


At this point we can visualize the mesh for inspection.
Capytaine has built-in methods for visualizing meshes (`fb.show`, and `fb.show_matplotlib`). 
When running outside a Notebook, these are interactive.  
The included WaveBot example also has a method for plotting the cross-section of the device. 

In [3]:
# fb.show_matplotlib()
# _ = wb.plot_cross_section(show=True)  # specific to WaveBot


#### Frequency and mesh check
We will analyze 50 frequencies with a spacing of 0.05 Hz. These frequencies will be used for the Fourier representation of both the wave and the desired PTO force in the pseudo-spectral problem. See the Theory section of the Documentation for more details on the pseudo-spectral problem formulation.

The `fb.minimal_computable_wavelength` parameter checks the mesh to determine the minimum wavelength that can be reliably computed using Capytaine. This warning is ignored here because the BEM results have been validated, but can be used as a guide for mesh refinement to ensure accurate BEM results.

In [4]:
df = 0.15
f1 = df
ifreq_start = 2
ifreq_end = 10 # 50

freq = wot.frequency(df, ifreq_end)[ifreq_start:]

min_computable_wavelength = fb.minimal_computable_wavelength
g = 9.81
min_period = 1/(df*ifreq_end)
min_wavelength = (g*(min_period)**2)/(2*np.pi)

if min_wavelength < min_computable_wavelength:
    print(f'Warning: Minimum wavelength in frequency spectrum ({min_wavelength}) is smaller'
         f' than the minimum computable wavelength ({min_computable_wavelength}).')


#### BEM
With our Capytaine floating body created, we can now run the Boundary Element Method solver in Capytaine to get the hydrostatic and hydrodynamic coefficients of our WEC object. This is wrapped into the `wecopttool.run_bem` function.

If you would like to save our BEM data to a NetCDF file for future use, see the `wecopttool.write_netcdf` function.

In [5]:
bem_data = wot.run_bem(fb, freq)


The resolution of the mesh 'WaveBot' of the body 'WaveBot_immersed' might be insufficient for the wavelength λ=6.94e-01.
This warning appears because the largest panel of this mesh has radius 1.01e-01 > λ/8.
The resolution of the mesh 'WaveBot' of the body 'WaveBot_immersed' might be insufficient for the wavelength λ=6.94e-01.
This warning appears because the largest panel of this mesh has radius 1.01e-01 > λ/8.


#### PTO
WecOptTool includes the `PTO` class to encompass all properties of the power take-off system of the WEC. Data wrapped into our `PTO` class will be used to help define our `WEC` object and optimization problem later.

To create an instance of the `PTO` class, we need:
- The kinematics matrix, which converts from the WEC degrees of freedom to the PTO degrees of freedom. The PTO extracts power directly from the WEC's heave in this case, so the kinematics matrix is simply the $1 \times 1$ identity matrix.
- The definition of the PTO controller. The `wecopttool.pto` submodule includes P, PI, and PID controller functions that can be provided to the `PTO` class and return the PTO force. However, we will be using an unstructured controller in this case, so we will set `None` for the controller.
- Any PTO impedance. We're only interested in mechanical power for this first problem, so we will leave this empty for now
- The non-linear power conversion loss (assumed 0% if `None`)
- The PTO system name, if desired

In [6]:
name = ["PTO_Heave",]
kinematics = np.eye(ndof)
controller = None
loss = None
pto_impedance = None
pto = wot.pto.PTO(ndof, kinematics, controller, pto_impedance, loss, name)


Now let's define the PTO forcing on the WEC and the PTO constraints. For our optimization problem, the constraints must be in the correct format for `scipy.optimize.minimize()`. We will enforce the constraint at 4 times more points than the dynamics (see Theory for why this is helpful for the pseudo-spectral problem).

In [8]:
# DEBUG
constraints = []


#### `WEC` creation
We are now ready to create the `WEC` object itself! Since we ran our BEM already, we can define the object using the `wecopttool.WEC.from_bem` function. If we saved our BEM data to a NetCDF file, we can also provide the path to that file instead of specifying the BEM `Dataset` directly.

In [9]:
wec = wot.WEC.from_bem(
    bem_data,
    constraints=constraints,
    friction=None,
    f_add=f_add,
)


Note: We might receive a warning regarding negative linear damping values. Per default, WecOptTool ensures that the BEM data does not contain non-negative damping values. If you would like to correct the BEM solution manually to a minimum damping value you can specify `min_damping`. 

### Waves
The wave environment must be specified as a 2-dimensional `xarray.DataArray` containing the complex amplitude (m). 
The two coordinates are the radial frequency ``omega`` (rad/s)  and the direction ``wave_direction`` (rad). 
The `wecopttool.waves` submodule contains functions for creating this `xarray.DataArray` for different types of wave environments. 

In this case we will use a regular wave with a frequency of 0.3 Hz and an amplitude of 0.0625 m. 
We will use the `wecopttool.waves.regular_wave` function. 

In [10]:
f1 = df
amplitude = 0 # 0.0625
wavefreq = 0.3
phase = 30
wavedir = 0
waves = wot.waves.regular_wave(f1, ifreq_end, wavefreq, amplitude, phase, wavedir, ifreq_start)


### Objective function
The objective function is the quantity (scalar) we want to optimize—in this case, the average mechanical power. The objective function is itself a function of the optimization state, the size of which we need to properly define our call to `scipy.optimize.minimize()`. The average mechanical power can be taken directly from the `PTO` object we created.

One technical quirk here: `nstate_opt` is one smaller than would be expected for a state space representing the mean (DC) component and the real and imaginary Fourier coefficients. This is because WecOptTool excludes the imaginary Fourier component of the highest frequency (the 2-point wave). Since the 2-point wave is sampled at multiples of $\pi$, the imaginary component is evaluated as $sin(n\pi); n = 0, 1, 2, ..., n_{freq}$, which is always zero. Excluding this component speeds up the optimization as the state space is reduced by one.

In [11]:
obj_fun = pto.mechanical_average_power
nstate_opt = wec.ncomponents


### Solve
We are now ready to solve the problem. WecOptTool uses `scipy.optimize.minimize` as its optimization driver, which is wrapped into `wecopttool.WEC.solve` for ease of use.

Note that the only required inputs for defining and solving the problem are: (1) the waves, (2) the objective function, and (3) the size of the optimization state. Optional inputs can be provided to control the optimization execution if desired, which we do here to change the default iteration maximum and tolerance. See `scipy.optimize.minimize` docs for more details.

To help the optimization we will scale the problem before solving it (see Documentation). WecOptTool allows you to scale the WEC dynamics state, your optimization state (in this case the Fourier coefficients for the PTO force), and the objective function separately. See the `wecopttool.WEC.solve()` function for more information.


Pay attention to the `Exit mode`: an exit mode of $0$ indicates a successful solution. For an easy problem (linear, single Dof, unconstrained, etc.) your iterations shouldn't need to exceed 100. If they do, try adjusting the scales by orders of magnitude, one at a time.

In [12]:
options = {'maxiter': 100}
# scale_x_wec = 1.0 # 1e1
# scale_x_opt = 1.0 # 1e-3
# scale_obj = 1.0 # 1e-2


scale_x_wec = 1.0
scale_x_opt = 1.0
scale_obj = 1.0

# results  = wec.solve(
problem  = wec.solve(
    waves,
    obj_fun,
    nstate_opt,
    # x_wec_0 = x_wec*1.1,
    # x_opt_0 = x_opt*0.9,
    optim_options=options,
    scale_x_wec=scale_x_wec,
    scale_x_opt=scale_x_opt,
    scale_obj=scale_obj,
    # use_grad = False,
    )

# opt_mechanical_average_power = results.fun
# print(f'Optimal average mechanical power: {opt_mechanical_average_power} W')


## DEBUG

In [13]:
## SOLVE!!!! ##
from scipy.optimize import minimize
import scipy; print(scipy.__version__)

optim_res = minimize(**problem)

1.11.4
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0004998016765365283
            Iterations: 8
            Function evaluations: 8
            Gradient evaluations: 8


In [14]:
problem.keys()

dict_keys(['fun', 'x0', 'method', 'constraints', 'options', 'bounds', 'callback', 'jac'])

In [15]:
problem['method']


'SLSQP'

In [16]:
problem['options']

{'maxiter': 100, 'disp': True}

In [17]:
problem['bounds'] == None

True

In [18]:
# SETUP
# idx = 3
# x_wec = np.zeros(wec.ncomponents); x_wec[idx*2] = 1; x_wec[idx*2+1] = 1; x_wec[0] = 1
# x_opt = np.zeros(wec.ncomponents); x_opt[idx*2] = 1; x_opt[idx*2+1] = 1; x_opt[0] = 1

# x = np.concatenate([x_wec, x_opt])

# t = wec.time

In [19]:
# Optimal
x_wec = np.array([
         -5.34590357e-05,
        # -1.44175178e-04, -1.69577759e-04,
        # -1.93732274e-05,  2.95446742e-04,
         9.23568964e-04,  -3.34575436e-04,
        -6.84667669e-05, -4.50653781e-04,
         9.28453882e-04,   7.75556889e-05,
         1.65461770e-01,  -2.25669377e-01,
         5.34106551e-04,   1.36036042e-04,
        -1.36430591e-04,  4.02670813e-04,
        -3.80444249e-05,  1.63276367e-05,
        -1.57437036e-04,
        ]);

x_opt = np.array([
        -1.30402770e+00,
        # -3.48316585e+00, -4.09789281e+00,
        # -4.67649366e-01,  6.92938769e+00,
         2.06482440e+01, -7.26660182e+00,
        -1.16610808e+00, -9.37008614e+00,
         1.74020260e+01,  2.51384525e+00,
         2.35013292e+03, -4.10306916e+03,
         7.42264452e+00,  3.45914541e+00,
        -3.14415012e+00,  4.46682294e+00,
        -4.50072846e-01, -6.55878884e-03,
        -1.16075834e+00,
        ]);

In [20]:
x = np.concatenate([x_wec, x_opt])

t = wec.time

In [21]:
# objetive function
f = problem['fun']
y = f(x)
y

ValueError: operands could not be broadcast together with shapes (32,) (36,) 

In [ ]:
# jacobian
f = problem['jac']
y = f(x)
plt.plot(y, 'o')

In [ ]:
# constraint (dynamics in residual form)
problem['constraints']

In [ ]:
f = problem['constraints'][0]['fun']
y = f(x)
plt.plot(t, y)

In [ ]:
f = problem['constraints'][0]['jac']
y = f(x)
plt.imshow(y, cmap="seismic")
plt.colorbar()
print(y.shape)

In [ ]:
f = problem['constraints'][0]['jac']
y = f(x) * x
plt.imshow(y, cmap="seismic")
plt.colorbar()

In [ ]:
# callback
f = problem['callback']
y = f(x)
y == None

In [ ]:
# x0 initial guess
plt.plot(problem['x0'], 'o')
plt.plot([16, 16], [-3, 3], 'k--')